# Kaggle link: https://www.kaggle.com/code/l1ghtsource/yappy-faces-clustering

In [1]:
state = 'clustering' # @[clustering, video_processing]

In [2]:
if state == 'video_processing':
    ! pip install dlib
    ! pip install face_recognition
    ! mkdir videos
    import os
    import cv2
    import time
    import tqdm
    import face_recognition

In [3]:
if state == 'clustering':
    ! pip install hdbscan
    import hdbscan
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 30.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.8
    Uninstalling Cython-3.0.8:
      Successfully uninstalled Cython-3.0.8


In [4]:
! pip install clickhouse_connect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.2/966.2 kB 11.3 MB/s eta 0:00:0000:010:01


In [5]:
import clickhouse_connect
import pandas as pd
from urllib.request import urlopen
from shutil import copyfileobj

In [6]:
df = pd.read_csv('/kaggle/input/yappy-csv-data/yappy_hackaton_2024_400k.csv')

# Функции для работы с БД Clickhouse

In [7]:
client = clickhouse_connect.get_client(host='91.224.86.248', 
                                       port=8123, 
                                       username=..., 
                                       password=...)


def load_from_csv(path, start, count):
    with open(path, encoding='utf-8') as f:
        text = f.read().split('\nhttps://cdn-st.rutubelist.ru')
    try:
        text = text[start + 1:start + count + 1]
    except IndexError:
        text = text[start + 1:]
    videos = list(map(lambda x: [start + x[0]] + x[1].split(','), enumerate(text)))
    return [[i[0], 'https://cdn-st.rutubelist.ru' + i[1], ','. join(i[2:])] for i in videos]


def download_to(url, path):
    with urlopen(url) as in_stream, open(str(path), 'wb') as out_file:
        copyfileobj(in_stream, out_file)

        
def push_data(data):
    client.insert('faces_embeddings', data, column_names=['id', 'face_emb', 'rank'])
    
    
def iteration_faces(fp, count, max_id=-1):
    if max_id == -1:
        max_id = client.query(f'SELECT max(id) FROM faces_embeddings').result_columns
        print('maxid', max_id)
        max_id = max_id[0][0] + 1
    videos = load_from_csv(fp, max_id, count)
    data = []
    for video in videos:
        fname = 'videos/' + str(video[0]) + '.mp4'
        url = video[1]
        download_to(url, fname)
        data += facefounder.process_file(fname, video[0])
    push_data(data)

# Класс для поиска лиц на видео

In [8]:
class FaceFounder():
    def __init__(self):
        pass

    def from_file(self, video_path):
        cap = cv2.VideoCapture(video_path, cv2.CAP_FFMPEG)
        images = self.get_frames(cap)
        unique = self.get_unique(images)
        return unique

    def get_frames(self, cap):
        frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        images = []
        if fps < 1:
            print('ERROR: problem reading video file: ')
        else:
            total_duration = (frameCount + fps - 1) // fps
            start_sec, end_sec = 0, total_duration
            interval = fps
            frames_idx = np.floor(np.arange(start_sec * fps, end_sec * fps, interval))
            ret = True

            for i, idx in enumerate(frames_idx):
                cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                images.append(frame)

        cap.release()
        return frame

    def get_frames_polnostju(self, fp):
        cap = cv2.VideoCapture(fp, cv2.CAP_FFMPEG)

        success, frame = cap.read()

        frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_duration = (frameCount + fps - 1) // fps

        frame_interval = max([2, total_duration / 6])

        images = []
        count = 0

        while success:
            if count % int(frame_interval * fps) == 0:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                images.append(frame)
            success, frame = cap.read()
            count += 1
        cap.release()

        return images

    def get_unique(self, images, video_id):
        face_locations = [face_recognition.face_locations(img) for img in images]

        face_encodings = []
        for i in range(len(images)):
            for f in face_recognition.face_encodings(images[i], face_locations[i]):
                face_encodings.append(f)

        diff_encodings = []
        enc_counter = []
        for e in face_encodings:
            found = False
            mask = face_recognition.compare_faces(diff_encodings, e)
            if True not in mask:
                diff_encodings.append(e)
                enc_counter.append(1)
            else:
                ind = mask.index(True)
                diff_encodings[ind] += e
                diff_encodings[ind] /= 2
                enc_counter[ind] += 1

        enc_pairs = [[diff_encodings[i], enc_counter[i]] for i in range(len(enc_counter))]
        enc_pairs.sort(key=lambda x: x[1])
        data = []
        for i, enc in enumerate(reversed(enc_pairs)):
            data.append([video_id, enc[0], i])

        return data

    def process_file(self, fp, vid):
        images = self.get_frames_polnostju(fp)
        return self.get_unique(images, vid)

# Запись извлеченной из видео информации в БД

In [ ]:
# facefounder = FaceFounder()

# start_id = 37500 # start id
# for i in tqdm.tqdm(range(100)): # num of iterations (1 iteration == 10 videos)
#     iteration_faces('/kaggle/input/yappy-csv-data/yappy_hackaton_2024_400k.csv', 10, start_id + (i * 10))

# Кластеризация (проводилась на 31000 видео)

In [9]:
df_faces = client.query_df(f'SELECT * FROM faces_embeddings')

embeddings = np.array(df_faces['face_emb'].tolist())

scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)

pca = PCA(n_components=50)
embeddings_reduced = pca.fit_transform(embeddings_scaled)

clusterer = hdbscan.HDBSCAN(min_cluster_size=10, core_dist_n_jobs=-1)

cluster_labels = clusterer.fit_predict(embeddings_reduced)
df_faces['cluster'] = cluster_labels

cluster_dict = df_faces.groupby('cluster')['id'].apply(list).to_dict()

for cluster, ids in cluster_dict.items():
    cluster_dict[cluster] = [df.iloc[id_].link for id_ in ids]

# Отсмотрел получившиеся кластеры и оставил адекватные

In [171]:
# дропаем астрологию, таро, рекламу странных лекарств и всякое такое
to_drop = [-1, 0, 1, 15, 17, 20, 25, 28, 32, 34, 35, 37, 38, 42, 48, 50, 53, 56, 59, 63, 65, 68, 70, 71, 75, 80, 82, 83, 86, 88, 90, 91, 93, 
           94, 95, 96, 99, 100, 101, 102, 104, 107, 108, 112, 114, 115, 116, 118, 125, 126, 128, 129, 130, 131, 132, 133, 134, 135, 137] 

In [166]:
mapping = {
    2: 'про фильмы',
    3: 'про игры',
    4: 'про телефоны',
    5: 'интересные факты',
    6: 'про уход за кожей',
    7: 'про историю Росиии и СССР',
    8: 'про бизнес',
    9: 'про здоровье',
    10: 'про корабли',
    11: 'про макияж',
    12: 'про нейросети',
    13: 'про арабский язык',
    14: 'про counter-strike',
    16: 'разнобой',
    18: 'про телефоны',
    19: 'про макияж',
    21: 'про алкоголь',
    22: 'интересные факты',
    23: 'про китайский язык',
    24: 'про мультфильмы',
    26: 'про спорт',
    27: 'про телефоны',
    29: 'про здоровье',
    30: 'про прически',
    31: 'про речь',
    33: 'про доту',
    36: 'про математику',
    39: 'про историю России',
    40: 'про рецепты',
    41: 'скетчи',
    43: 'про gta',
    44: 'про бизнес',
    45: 'про макияж',
    46: 'про озвучку',
    47: 'про одежду',
    49: 'про макияж',
    51: 'про факты о животных',
    52: 'про психологию',
    54: 'про одежду',
    55: 'интересные факты',
    57: 'про макияж',
    58: 'разнобой',
    60: 'про математику',
    61: 'про макияж',
    62: 'про макияж',
    64: 'про рецепты',
    66: 'про жизнь',
    67: 'про ЕГЭ по истории',
    69: 'про историю',
    72: 'про рецепты',
    73: 'интересные факты',
    74: 'про аниме',
    76: 'про телефоны',
    77: 'про телефоны',
    78: 'про сериалы',
    79: 'про автомобили',
    81: 'разнобой',
    84: 'про фильмы и актеров',
    85: 'про одежду',
    87: 'про математику',
    89: 'про макияж',
    92: 'про одежду',
    97: 'про аниме',
    98: 'про макияж',
    103: 'разнобой',
    105: 'про психологию',
    106: 'папич',
    109: 'про поэтов',
    110: 'про игры',
    111: 'про технику',
    113: 'про русский язык',
    117: 'про геймдев',
    119: 'про мотоциклы',
    120: 'про речь',
    121: 'про игры',
    122: 'про макияж',
    123: 'про русский язык',
    124: 'про футбол',
    127: 'про восточные страны',
    136: 'про рецепты'
}

In [184]:
mapping.keys()

dict_keys([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 18, 19, 21, 22, 23, 24, 26, 27, 29, 30, 31, 33, 36, 39, 40, 41, 43, 44, 45, 46, 47, 49, 51, 52, 54, 55, 57, 58, 60, 61, 62, 64, 66, 67, 69, 72, 73, 74, 76, 77, 78, 79, 81, 84, 85, 87, 89, 92, 97, 98, 103, 105, 106, 109, 110, 111, 113, 117, 119, 120, 121, 122, 123, 124, 127, 136])

In [167]:
df_clusters = pd.DataFrame(df_faces.groupby('cluster')['id'].apply(list))

In [168]:
df_clusters['links'] = df_clusters['id'].apply(lambda ids: [df.iloc[id_].link for id_ in ids])

In [169]:
df_clusters = df_clusters.reset_index()

In [172]:
df_clusters = df_clusters[~df_clusters['cluster'].isin(to_drop)]

In [173]:
df_clusters['name'] = [mapping[id_] for id_ in df_clusters.cluster]

In [269]:
df_clusters

,cluster,id,links,name
3,2,"[42372, 2678, 5185, 7520, 18003, 18070, 27340,...",[https://cdn-st.rutubelist.ru/media/91/f5/1f21...,про фильмы
4,3,"[1478, 2029, 2673, 2948, 3811, 3999, 7777, 821...",[https://cdn-st.rutubelist.ru/media/4b/3e/0d6b...,про игры
5,4,"[39666, 2301, 5800, 6051, 6106, 12246, 13507, ...",[https://cdn-st.rutubelist.ru/media/20/1a/93ea...,про телефоны
6,5,"[4603, 9313, 14785, 16589, 26235, 26305, 27428...",[https://cdn-st.rutubelist.ru/media/3f/c9/0803...,интересные факты
7,6,"[42643, 298, 535, 2220, 6774, 7864, 8887, 9145...",[https://cdn-st.rutubelist.ru/media/9d/e7/2365...,про уход за кожей
...,...,...,...,...
123,122,"[45609, 1485, 2025, 2912, 3569, 4623, 4772, 52...",[https://cdn-st.rutubelist.ru/media/0e/5a/0b3d...,про макияж
124,123,"[42394, 45985, 181, 1311, 3095, 3230, 4497, 81...",[https://cdn-st.rutubelist.ru/media/e3/cb/66bd...,про русский язык
125,124,"[42545, 45881, 45891, 345, 1307, 1456, 3657, 5...",[https://cdn-st.rutubelist.ru/media/66/cc/c1bb...,про футбол
128,127,"[42506, 690, 1419, 3186, 3843, 5272, 5554, 594...",[https://cdn-st.rutubelist.ru/media/df/ec/efab...,про восточные страны


In [270]:
df_clusters.to_csv('faces-31000-videos-filtered.csv')